<style>.md-sidebar--secondary {display: none !important;}</style>

# Retrospective Analysis

In this example, we will demonstrate some analysis that is reading back 
results of prior completed simulation run.  This is possible because
PassengerSim can be configured to write out quite a bit of data to a
SQLite database while running.  Much of this output is optional, as writing
out intermediate results can create very large database files and impose a
significant runtime penalty on the simulation, but having the ability to 
do so is advantageous for research and development purposes.

In [ ]:
import passengersim as pax
pax.versions()

In [ ]:
# This cell creates the two on-disk database files.  
# It is hidden in the rendered documentation as this code is not the focus of this demo page.
import os
from passengersim.utils.tempdir import TemporaryDirectory
tempdir = TemporaryDirectory()

cfg = pax.Config.from_yaml("network/08-untrunc-em.yaml")
cfg.simulation_controls.num_trials = 1
cfg.simulation_controls.num_samples = 900
cfg.simulation_controls.show_progress_bar = False
cfg.db.filename = tempdir.joinpath("untruncated.sqlite")
pax.Simulation(cfg).run();

cfg = pax.Config.from_yaml("network/05-emsrb.yaml")
cfg.simulation_controls.num_trials = 1
cfg.simulation_controls.num_samples = 900
cfg.simulation_controls.show_progress_bar = False
cfg.db.filename = tempdir.joinpath("simple.sqlite")
pax.Simulation(cfg).run();

os.chdir(tempdir.name)

We can load in `SummaryTables` objects directly from the database created
during a simulation run (assuming it was saved to disk).  For this demo,
we have created two small such databases by running two different models
with on-disk database files, and then discarding the results (for now).
We can then load summary tables of results from these two simulations
by using the `SummaryTables.from_sqlite` constructor.  This command will
run queries against the database to reconstruct the same summary table
objects we would have gotten as the output from running the simulation.

In [ ]:
simple = pax.SummaryTables.from_sqlite("simple.sqlite")
untrunc = pax.SummaryTables.from_sqlite("untruncated.sqlite")

In [ ]:
from passengersim.contrast import Contrast

comps = Contrast(
    Simple=simple, Untruncated=untrunc
)

All the same visualizations available in our prior demonstrations are 
also available for use with the pre-computed results.
Since we are not actually running the simulation again, this analysis can 
even be done by users who don't have (or need) access to the PassengerSim
core simulation code.

In [ ]:
comps.fig_carrier_revenues()

In [ ]:
comps.fig_carrier_load_factors()

In [ ]:
comps.fig_fare_class_mix()

In [ ]:
comps.fig_bookings_by_timeframe(by_carrier="AL1", by_class=True, source_labels=True)

The results of the simulation are stored in database tables.  We can write
queries using typical SQLite syntax against the database directly if we like.
For convenience, the SQLite database connection used to populate the summary
tables is available as the `cnx` attribute of the SummaryTables object, and 
we can use its `dataframe` method to run arbitrary queries and get the results
as a pandas DataFrame.

In [ ]:
simple.cnx.dataframe("""
SELECT
  sample, auth, sold, forecast_mean, forecast_stdev
FROM 
  leg_bucket_detail 
WHERE
  flt_no = 101
  AND rrd = 21
  AND name = 'Y2'
  AND sample >= 100
LIMIT 10
""")